In [1]:
import sys
import os
this_dir = ".."
if this_dir not in sys.path:
    sys.path.append(this_dir)

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import RNN, train_one_epoch, eval_model


import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence, pack_sequence

from sklearn.model_selection import ParameterGrid, ParameterSampler

from variational_rnn import VariationalRNN
from locked_dropout import LockedDropout
from weight_drop import WeightDrop

import numpy as np

from bayes_opt import BayesianOptimization

In [2]:
# Reproducibility
np.random.seed(546)
torch.manual_seed(546)
torch.cuda.manual_seed(546)

In [3]:
grouping = 'ccs'
batch_size=64

In [4]:
dataset = DiagnosesDataset('../data/model_data.json',grouping)
test_size = 0.15
eval_size=0.15
eval_size_corrected = eval_size/(1-test_size)

whole_train_dataset,test_dataset = split_dataset(dataset,test_size)
train_dataset, val_dataset = split_dataset(whole_train_dataset,eval_size_corrected)

len(whole_train_dataset)
len(train_dataset)
len(val_dataset)
len(test_dataset)

whole_train_dataset = DataLoader(whole_train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

6375

5250

1125

1124

In [5]:
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
hidden_size = 100
num_layers = 2
n_labels = input_size
model_type = 'RNN'

In [11]:
dropouti = 0.2
dropoutw = 0.3
dropouto = 0.3

In [12]:
model = VariationalRNN(input_size=input_size,
                       hidden_size=hidden_size,
                       n_labels=n_labels,
                       num_layers=num_layers,
                       rnn_type=model_type,
                       dropouti=dropouti,
                       dropoutw=dropoutw,
                       dropouto=dropouto
                      )

In [11]:
opt = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
epochs = 15

In [12]:
losses = []
for epoch in range(epochs):
    loss = train_one_epoch(model,train_dataloader,epoch,criterion,opt)
    losses.append(loss)
    print(f'epoch: {epoch+1} | loss: {loss}')

epoch: 1 | loss: 0.45540750244047107
epoch: 2 | loss: 0.27043653647540383
epoch: 3 | loss: 0.18126442988948427
epoch: 4 | loss: 0.13168226220679852
epoch: 5 | loss: 0.10306270792738324
epoch: 6 | loss: 0.08516193667693042
epoch: 7 | loss: 0.07417724387998534
epoch: 8 | loss: 0.06439795336560601
epoch: 9 | loss: 0.058741568199758384
epoch: 10 | loss: 0.05428173068405524
epoch: 11 | loss: 0.050321973883776676
epoch: 12 | loss: 0.04807782656687791
epoch: 13 | loss: 0.04519118809427434
epoch: 14 | loss: 0.04257787666023838
epoch: 15 | loss: 0.04166140047277993


In [20]:
def train_multiple_epochs(**kwargs):
    n_epochs = kwargs.get('n_epochs',15)
    
    
    model = VariationalRNN(kwargs.get('input_size'),
                           kwargs.get('hidden_size'),
                           kwargs.get('n_labels'),
                           kwargs.get('num_layers'),
                           kwargs.get('rnn_type'),
                           kwargs.get('dropoutii'),
                           kwargs.get('dropoutww'),
                           kwargs.get('dropoutoo'))
    
    opt = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    for epoch in range(n_epochs):
        train_one_epoch(model,train_dataloader,epoch,criterion,opt)
    
    res = eval_model(model,val_dataloader,dataset,criterion,'all','eval',False,'last adm',30)
    return res['last adm']['recall30']['mean']
func = lambda dropoutii,dropoutww,dropoutoo: train_multiple_epochs(n_epochs=10,
                                                                input_size=input_size,
                                                                hidden_size=hidden_size,
                                                                n_labels=n_labels,
                                                                num_layers=num_layers,
                                                                rnn_type=model_type,
                                                                dropoutii=dropoutii,
                                                                dropoutww=dropoutww,
                                                                dropoutoo=dropoutoo)

In [13]:
# bayesian optimization

## bounds of parameter search
pbounds = {'dropoutii': (0.1,0.8), 
           'dropoutww': (0.1,0.8),
           'dropoutoo': (0.1,0.8)
          }

## parameter search
optimizer = BayesianOptimization(
    f=func,
    pbounds=pbounds,
    random_state=1,
)
optimizer.maximize(init_points=5,n_iter=15)

|   iter    |  target   | dropoutii | dropoutoo | dropoutww |
-------------------------------------------------------------
|  1        |  0.6002   |  0.3919   |  0.6042   |  0.1001   |
|  2        |  0.5985   |  0.3116   |  0.2027   |  0.1646   |
|  3        |  0.6059   |  0.2304   |  0.3419   |  0.3777   |
|  4        |  0.5755   |  0.4772   |  0.3934   |  0.5797   |
|  5        |  0.5895   |  0.2431   |  0.7147   |  0.1192   |
|  6        |  0.5383   |  0.3631   |  0.6529   |  0.6421   |
|  7        |  0.6168   |  0.1      |  0.1      |  0.4499   |
|  8        |  0.5568   |  0.1      |  0.1      |  0.8      |
|  9        |  0.5687   |  0.2593   |  0.2089   |  0.7631   |
|  10       |  0.5874   |  0.6084   |  0.61     |  0.2363   |
|  11       |  0.6287   |  0.1      |  0.1      |  0.242    |
|  12       |  0.5682   |  0.7688   |  0.5648   |  0.4295   |


KeyboardInterrupt: 

In [16]:
#best_params_1st_run = optimizer.max
#best_params_1st_run

best_params = optimizer.max
best_params

{'target': 0.6287178853774116,
 'params': {'dropoutii': 0.1,
  'dropoutoo': 0.1,
  'dropoutww': 0.2419500529180941}}